# Introduction
We've observed that the bounds for a lot of the reactions are limited to a max/min of +/-20. To allow unbiased flux distributions, we should make the bounds max and min +/-1000 (or 0 where irreversible). Then we should constrain what is fed into the model to get better, non-biased predicitions.

This notebook will write a script to modify the bounds of all reactions automatically.

__NOTE: this should only be implemented when all reactions are mass balanced & The medium problem has been fixed too & sugar conversion part__

In [1]:
import cameo
import pandas as pd
import cobra.io
import escher

In [2]:
model = cobra.io.read_sbml_model('../model/p-thermo.xml')

In [4]:
#original biomass prediction
model.optimize().objective_value

0.48302692481794285

In [5]:
#first just change all upper bounds
for rct in model.reactions:
    if rct.id in 'EX_glc__D_e':
        continue
    else:
        upper = rct.upper_bound
        if upper == 0:
            continue
        elif upper == 1000:
            continue
        elif upper == 20:
            rct.upper_bound = 1000
        elif upper == 10000:
            rct.upper_bount = 1000
        else: 
            print (rct.id,':', upper)

In [6]:
#then change all lower bounds
for rct in model.reactions:
    if rct.id in 'EX_glc__D_e':
        continue
    else:
        lower = rct.lower_bound
        if lower == 0:
            continue
        elif lower == -1000:
            continue
        elif lower == -20:
            rct.lower_bound = -1000
        elif lower == -10000:
            rct.lower_bount = -1000
        else: 
            print (rct.id,':', lower)

The glucose exchange (as default carbon source) should be limiting here, and so should be fixed at for e.g. 10. 

In [7]:
model.reactions.EX_glc__D_e.bounds = (-10,0)

In [9]:
#Check NGAM (ATPM) is also still fixed at 1.53. 

In [15]:
#save&commit
cobra.io.write_sbml_model(model,'../model/p-thermo.xml')

Interestingly, you get a significant increase in biomass prediction. This is likely because we will have erroneous (energy or carbon) generating cycles in the model. Hopefully these will be resolved with mass balance and looking into it further. 

Straight from analyzing the new fluxes, you can see that there is a cycle being formed between F1P, to fdp, to DHAP+G3P and around. Currently, i think this is possible because we have so many reactions that just form ATP where they shouldnt, and so this is not a limiting metabolite right now. I will open an issue about this, and hope that when mass balancing and energetics is fixed this is resolved, or then we take a deeper look into how to fix this. 

In [10]:
#look at effect on biomass
model.optimize()

,fluxes,reduced_costs
IDPh,0.000000,-6.026184e-03
PYRACTT,0.000000,-4.336809e-19
CAT,0.000000,0.000000e+00
PDHam1hi,0.000000,-1.084202e-19
CCP,-80.634403,-0.000000e+00
...,...,...
EX_dglcn5_e,0.000000,-9.271053e-05
EX_mdgp_e,0.000000,-3.430290e-03
EX_tag__D_e,0.000000,-6.860579e-03
EX_amylose_e,0.000000,-1.705874e-02


In [11]:
solution = model.optimize()

In [13]:
model.metabolites.fdp_c.summary()

,,PERCENT,FLUX,REACTION_STRING
RXN_STAT,ID,,,
PRODUCING,FRUK,100.0,1000.0,atp_c + f1p_c --> adp_c + fdp_c + 2.0 h_c
CONSUMING,FBA,100.0,1000.0,fdp_c <=> dhap_c + g3p_c


In [14]:
model.metabolites.f1p_c.summary()

,,PERCENT,FLUX,REACTION_STRING
RXN_STAT,ID,,,
PRODUCING,FBA2,100.0,1000.0,f1p_c <=> dhap_c + g3p_c
CONSUMING,FRUK,100.0,1000.0,atp_c + f1p_c --> adp_c + fdp_c + 2.0 h_c


In [12]:
escher.Builder('e_coli_core.Core metabolism', reaction_data=solution.fluxes.to_dict())

Builder(reaction_data={'IDPh': 0.0, 'PYRACTT': 0.0, 'CAT': 0.0, 'PDHam1hi': 0.0, 'CCP': -80.63440250408348, 'H…

In [ ]:
#save & commit:
cobra.io.write_sbml_model(model,'../model/p-thermo.xml')